In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
import os 
path_dir = './Final_data/one_week/'
file_list = os.listdir(path_dir) 

In [ ]:
del file_list[26] 
len(file_list) 

In [ ]:
df_list = []
cust_ip_list = [] 

for i in file_list:
    tmp_df = pd.read_parquet(path_dir+i, engine='pyarrow')
    tmp_df['stamp_inserted'] = pd.to_datetime(tmp_df['stamp_inserted'], format='%Y%m%d%H%M%S')
    tmp_df.event.fillna(0, inplace=True)
    df_list.append(tmp_df)
    c_ip = tmp_df.iloc[0].cust_ip_no 
    cust_ip_list.append(c_ip)

In [ ]:
# 디도스 + 전체 시간이 있는 데이터 
e_data = []

for n, df in enumerate(df_list):
    if len(df) >= 10080 and 1 in df.event.values :
        if 1 in df.event.values:
            df = df.drop_duplicates(['stamp_inserted'], keep='first')
            if len(df) >= 10080:
                df.reset_index(drop=True, inplace=True) 
                e_data.append(df)

In [ ]:
iv = ['bytes', 'pkts',
       'sip_entropy', 'dip_entropy', 'spt_entropy', 'dpt_entropy',
       'proto_entropy', 'tcp_flags_entropy', 'icmp_ratio', 'tcp_ratio',
       'udp_ratio', 'domestic_ratio', 'in_ratio', 'inner_ratio']
iv1 = ['bytes', 'pkts',
       'sip_entropy', 'dip_entropy', 'spt_entropy', 'dpt_entropy',
       'proto_entropy', 'tcp_flags_entropy', 'icmp_ratio', 'tcp_ratio',
       'udp_ratio', 'domestic_ratio', 'in_ratio', 'inner_ratio', 'cust_ip_no', 'event']
features = len(iv)


In [ ]:
len(e_data) 

In [ ]:
# IP별 정규화
from sklearn.preprocessing import MinMaxScaler
import joblib

X_data = []
y_data = [] 
for i in e_data:
    scaler = MinMaxScaler(feature_range=(-1,1))
    y_data.append(i[['cust_ip_no', 'event']].values)
    file_name = 'scaler_ip_{}.pkl'.format(i['cust_ip_no'].values[0]) 
    
    i = scaler.fit_transform(i[iv].values) 
    X_data.append(i)
    
    joblib.dump(scaler, file_name) 
    
# unseen data의 경우 scaler.partail_fit(new_data)

In [ ]:
scaler = joblib.load('scaler_ip_8879.pkl') 

In [ ]:
scaler

In [ ]:
new_data = []
for x, y in zip(X_data, y_data):
    data_ = np.concatenate((x, y), axis=1) 
    data_ = pd.DataFrame(data_, columns=iv1) 
    print(data_.iloc[0,-2])
    new_data.append(data_)  

In [ ]:
----몇 개씩 묶어서 해볼 때----
# train_X_ae = np.concatenate((train_X_ae1, train_X_ae5, train_X_ae6), axis=0)
# train_y_ae = np.concatenate((train_y_ae1, train_y_ae5, train_y_ae6), axis=0)
# test_X_ae = np.concatenate((test_X_ae1, test_X_ae5, test_X_ae6), axis=0)
# test_y_ae = np.concatenate((test_y_ae1, test_y_ae5, test_y_ae6), axis=0)


In [ ]:
from tqdm import tqdm

seq_time = 5

normal_X = [] 
normal_y = []
abnormal_X = [] 
abnormal_y = []


for j in tqdm(range(len(new_data))):
    normal_data = pd.DataFrame(columns=iv)
    normal_label = []
    abnormal_data = pd.DataFrame(columns=iv)
    abnormal_label = []
    for i in range(len(new_data[j]) - seq_time):        
        input_x = new_data[j].iloc[i:i+seq_time, :-2]
        label = new_data[j].iloc[i+seq_time-1, -1]
        ip_NO = new_data[j].iloc[i+seq_time-1, -2]  
        
        if label == 0:
            normal_data = pd.concat([normal_data, input_x], axis=0)
            ip_label_set = (ip_NO, label)            
            normal_label.append(ip_label_set)
            
        else :
            abnormal_data = pd.concat([abnormal_data, input_x], axis=0)
            ip_label_set = (ip_NO, label) 
            abnormal_label.append(ip_label_set)
            
    normal_X.append(normal_data)  #IP별 9개의 정상 데이터
    normal_y.append(normal_label) 
    if len(abnormal_data) != 0:
        abnormal_X.append(abnormal_data)
        abnormal_y.append(abnormal_label)   



In [ ]:
ip_idx = 0
X_normal = normal_X[ip_idx].values.reshape(-1, 5, 14) 
X_abnormal = abnormal_X[ip_idx].values.reshape(-1, 5, 14) 

from sklearn.model_selection import train_test_split
train_X_ae, test_X_ae, train_y_ae, test_y_ae = train_test_split(X_normal, normal_y[ip_idx], test_size=0.1, shuffle=True)

print(train_X_ae.shape) 
print(test_X_ae.shape) 


# test set에 일부 정상 값을 함께 넣어줌
test_X_ae= np.concatenate((test_X_ae, X_abnormal), axis=0)
test_y_ae = np.concatenate((test_y_ae, abnormal_y[ip_idx]), axis=0) 
 

In [ ]:
test_y_df = pd.DataFrame(test_y_ae, columns=['ip','label'])
train_y_df = pd.DataFrame(train_y_ae, columns=['ip','label']) 
test_y_df.head() 
train_y_df.head() 

In [ ]:
cur_ip = int(test_y_df.ip[0])
cur_ip

In [ ]:
ip8879 = train_X_ae.reshape(-1, 14)

In [ ]:
ip8879 = pd.DataFrame(ip8879, columns=iv)
ip8879

# 1. CAE 

In [ ]:
from tensorflow.keras import Model, models, optimizers, utils, layers
from keras.layers import LSTM, Dense, Activation, Conv1D, Dropout, Conv1DTranspose, BatchNormalization
from keras.models import load_model
import keras.backend as K

In [ ]:
rmsprop = optimizers.RMSprop(lr=0.0001) 
adam = optimizers.Adam(lr=0.0001)
kernel_s = 3
seq_time = 5

In [ ]:
#del cnn_ae

In [ ]:
K.clear_session()
cnn_ae = models.Sequential([
    layers.Input(shape=(seq_time, features)),
    layers.Conv1D(filters=14, kernel_size=kernel_s, strides=1),
    layers.BatchNormalization(),
    Activation('relu'),
    layers.Conv1D(filters=16, kernel_size=kernel_s, padding='same', strides=1),
    layers.BatchNormalization(), 
    Activation('relu'),
    layers.Conv1D(filters=32, kernel_size=kernel_s, padding='same', strides=1),
    layers.Conv1DTranspose(filters=32, kernel_size=kernel_s, padding='same', strides=1),
    layers.BatchNormalization(),
    Activation('relu'),
    layers.Conv1DTranspose(filters=16, kernel_size=kernel_s, padding='same', strides=1),
    layers.BatchNormalization(),
    Activation('relu'),
    layers.Conv1DTranspose(filters=14, kernel_size=kernel_s, padding='same'),
]) 

cnn_ae.compile(optimizer=adam, loss='mse') 
cnn_ae.summary()

In [ ]:
hist = cnn_ae.fit(train_X_ae, train_X_ae, epochs=50, validation_split=0.1, batch_size=16, shuffle=True, verbose=2)

In [ ]:
cnn_ae.save('./Model/model_ip_{}'.format(cur_ip))  

In [ ]:
# import plotly
# import plotly.graph_objects as go
# plotly.offline.init_notebook_mode(connected=True)

# loss = hist.history['loss']
# val_loss = hist.history['val_loss']

# fig = go.Figure() 
# fig.add_trace(go.Scatter(y=loss, mode='lines', name='loss'))
# fig.add_trace(go.Scatter(y=val_loss, mode='lines', name='val_loss'))
# fig.update_layout(height=400, width=600, title='Mix data: Loss of Convolutional AE')
# fig.show() 


In [ ]:
pred_x = cnn_ae.predict(test_X_ae).reshape(test_X_ae.shape[0], -1)
real = test_X_ae.reshape(test_X_ae.shape[0], -1)
mse = np.log(np.mean(np.power(real - pred_x, 2), axis=1))
test_df_error = pd.DataFrame({'reconstruction_error': mse, 'Label': test_y_df.label})


In [ ]:
test_df_error

In [ ]:
test_ddos = test_df_error[test_df_error.Label==1]
test_normal = test_df_error[test_df_error.Label==0] 
test_normal.describe() 

In [ ]:
pred_x = cnn_ae.predict(train_X_ae).reshape(train_X_ae.shape[0], -1)
real = train_X_ae.reshape(train_X_ae.shape[0], -1) 
mse = np.log(np.mean(np.power(real - pred_x, 2), axis=1))
train_df_error = pd.DataFrame({'reconstruction_error': mse, 'Label': train_y_df.label})

In [ ]:
thr = train_df_error.describe()
error = sorted(train_df_error.reconstruction_error.values, reverse=True)

up_to_99_5 = int(round(len(error) * 0.005,0)) 
up_to_99_5

threshold = error[up_to_99_5]
print(thr)
print(threshold) 

In [ ]:
ddos_num = len(test_ddos)
if ddos_num > 10 :
    random_count = int(round(ddos_num*0.1, 0)) 
    print('이용가능한 디도스 데이터는 {}개 입니다.'.format(random_count))
    sampled_idx = list(np.random.randint(ddos_num, size=(random_count)))
    sampled_threshold = [test_ddos.iloc[i,0] for i in sampled_idx]
    difference = [i- threshold for i in sampled_threshold]
    print(difference) 

In [ ]:
safe_line = error[0] - threshold
safe_line

In [ ]:
error[0] 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
pred_y = [1 if e > threshold else 0 for e in test_df_error['reconstruction_error'].values]

print(confusion_matrix(test_df_error.Label, pred_y)) 

In [ ]:
m = confusion_matrix(test_df_error.Label, pred_y)

In [ ]:
m[1]

In [ ]:
# 원래 임계치로 했을 때 
from sklearn.metrics import classification_report, confusion_matrix, auc, roc_curve

if difference != 0:
    if min(difference)> safe_line:
        pred_y = [ 1 if e> threshold else 0 for e in test_df_error['reconstruction_error'].values]
        print('새로운 임계치 {}로 갱신되었습니다.'.format(threshold))
    else:
        pred_y = [ 1 if e> threshold else 0 for e in test_df_error['reconstruction_error'].values]
else: 
    pred_y = [ 1 if e> threshold else 0 for e in test_df_error['reconstruction_error'].values]
        
print(classification_report(test_df_error.Label, pred_y)) 
print(confusion_matrix(test_df_error.Label, pred_y)) 

In [ ]:
# 갱신 규칙을 적용했을 때 
if difference != 0:
    if min(difference)> safe_line:
        threshold = error[0] 
        pred_y = [ 1 if e> threshold else 0 for e in test_df_error['reconstruction_error'].values]
        print('새로운 임계치 {}로 갱신되었습니다.'.format(threshold))
    else:
        pred_y = [ 1 if e> threshold else 0 for e in test_df_error['reconstruction_error'].values]
else: 
    pred_y = [ 1 if e> threshold else 0 for e in test_df_error['reconstruction_error'].values]
        
print(classification_report(test_df_error.Label, pred_y)) 
print(confusion_matrix(test_df_error.Label, pred_y)) 

In [ ]:
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(test_df_error.Label, pred_y)
roc_auc = auc(fpr, tpr)

plt.figure() 
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import plotly.figure_factory as ff

data = [train_df_error.reconstruction_error, test_ddos.reconstruction_error, test_normal.reconstruction_error]
group = ['normal of train', 'ddos of test', 'normal of test'] 
fig = ff.create_distplot(data, group, bin_size=[.3,.3,.3])
fig.show() 

In [ ]:
import plotly.figure_factory as ff

data = [train_df_error.reconstruction_error, test_normal.reconstruction_error]
group = ['normal of train', 'normal of test'] 
fig = ff.create_distplot(data, group, bin_size=[.3,.3])
fig.show() 

In [ ]:
len(train_df_error[train_df_error.reconstruction_error>-6])

In [ ]:
len(train_df_error[train_df_error.reconstruction_error<-6])

In [ ]:
141/142

In [ ]:
2/(1/0.892 + 1/0.992)